# Download Subdirectories From FTP

#### Updated: Aug 9, 2023

#  

Download subdirectories from FTP one at a time. Record download in a 'download_log'. Move downloaded subdirectories from '01_downloading' folder to '02_downloaded' folder. 

In [1]:
import pandas as pd
import os
import shutil
import time
from datetime import datetime
import ftplib
from random import randrange

In [2]:
rootDir = '' #The file path to the root directory for your project

#Within your root directory, create the following folders:
# 01_downloading
# 02_downloaded
# 03_processing
# 04_processed
# 04a_unprocessed
# 05_result
# 06_log

downloadingDir = rootDir + '01_downloading/'
downloadedDir = rootDir + '02_downloaded/'
logDir = rootDir + '06_log/'

In [3]:
HOSTNAME = "data.asc-csa.gc.ca"
USERNAME = "anonymous"
PASSWORD = #**users must enter their email address**

# Connect FTP Server
ftp = ftplib.FTP(HOSTNAME, USERNAME, PASSWORD)
print('Connected to ftp server: ' + HOSTNAME)

ftp_rootpath = 'users/OpenData_DonneesOuvertes/pub/Alouette-ISIS/Alouette-1/'

Connected to ftp server: donnees-data.asc-csa.gc.ca


#  

#### Functions:

In [4]:
def move_images(old_dir, new_dir, directory, subdirectory, copy_to_other_drive=False):
    oldDir = old_dir + directory + '/' + subdirectory + '/'
    newDir = new_dir + directory + '/' + subdirectory + '/'
    os.makedirs(newDir, exist_ok=True)
    
    if copy_to_other_drive:
        for file in os.listdir(oldDir):
            shutil.copyfile(oldDir+file, newDir+file)
    else:
        for file in os.listdir(oldDir):
            os.rename(oldDir+file, newDir+file)
    
    shutil.rmtree(old_dir + directory + '/' + subdirectory + '/')
    if len(os.listdir(old_dir + directory + '/')) == 0:
        shutil.rmtree(old_dir + directory + '/')

In [5]:
def draw_random_subdir(directory, subdir_list, logDir):
    
    subdirectory = subdir_list[randrange(len(subdir_list))]
    
    #Check randomly-selected directory and subdirectory against the 'download_log'
    if os.path.exists(logDir + 'download_log.csv'):
        df_log = pd.read_csv(logDir + 'download_log.csv')
        df_search = df_log.loc[(df_log['Directory'] == directory) & (df_log['Subdirectory'] == subdirectory)]
        if len(df_search) > 0:
            print(directory + '/' + subdirectory + ' already downloaded!')
            return ''
        else:
            return subdirectory
    else:
        return subdirectory

#  

#### Download a random subdirectory by FTP:

In [6]:
#Draw random Directory
ftp.cwd(ftp_rootpath)
directory_list = ftp.nlst()
directory = directory_list[randrange(len(directory_list))]

ftp.cwd(directory)
subdir_list = ftp.nlst()

#Randomly draw Directory and Subdirectory
subdirectory = ''
while (subdirectory == ''):
    subdirectory = draw_random_subdir(directory=directory, subdir_list=subdir_list, logDir=logDir)

In [7]:
saveDir = downloadingDir + directory + '/' + subdirectory + '/'
os.makedirs(saveDir, exist_ok=True)
ftp.cwd(ftp_rootpath + '/' + directory + '/' + subdirectory + '/')

'250 Directory successfully changed.'

In [8]:
start = time.time()
n_dl = len(ftp.nlst())
print('Downloading ' + directory + '/' + subdirectory + '/ subdirectory ('  + str(n_dl) + ' images)')
for file in ftp.nlst():
    local_file = open(saveDir + file, "wb")
    ftp.retrbinary("RETR " + file, local_file.write)
    local_file.close()
    #print('Downloaded: ' + file)
end = time.time()
t = end - start
print('Download time for subdirectory: ' + str(round(t/60, 1)) + ' min')

Download time for subdirectory: 4.4 min


Record subdirectory name in download_log:

In [9]:
df_result = pd.DataFrame({
    'Directory': directory,
    'Subdirectory': subdirectory,
    'Images_downloaded': n_dl,
    'Download_time': t,
    'Download_timestamp': datetime.fromtimestamp(end)
}, index=[0])
df_result

Roll Subdirectory  Images_downloaded  Download_time  \
0  R014207821     3363-A38                245     264.304774   

          Download_timestamp  
0 2022-12-07 19:56:31.865868

In [10]:
if os.path.exists(logDir + 'download_log.csv'):
    df_log = pd.read_csv(logDir + 'download_log.csv')
    df_update = pd.concat([df_log, df_result], axis=0, ignore_index=True)
    df_update.to_csv(logDir + 'download_log.csv', index=False)
else:
    df_result.to_csv(logDir + 'download_log.csv', index=False)

Backup 'download_log' (10% of the time):

In [11]:
if randrange(10) == 7:
    df_log = pd.read_csv(logDir + 'download_log.csv')
    datetime_str = datetime.now().strftime("%Y%m%d_%Hh%M")
    os.makedirs(logDir + 'backups/', exist_ok=True)
    df_log.to_csv(logDir + 'backups/' + 'download_log-' + datetime_str + '.csv', index=False)

Move fully downloaded subdirectory to 02_processing folder:

In [12]:
move_images(old_dir=downloadingDir, new_dir=downloadedDir, directory=directory, subdirectory=subdirectory)